# Практика

Скоринг учащихся (следующий семестр на основе текущего, количество 2/3/4/5)

## Загрузка библиотек

In [1]:
!pip install catboost
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import Pool, CatBoost, CatBoostRegressor, cv

## Предобработка данных

In [3]:
pre_data = pd.read_excel('Успеваемость_01.xlsx')

In [4]:
pre_data.to_csv('/content/new_data.csv', index=False)

In [5]:
df_new = pd.read_csv('new_data.csv')

In [6]:
df_new.head(1)

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость)
0,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,Инженерная компьютерная графика,Хорошо,Хорошо


In [7]:
df_new = df_new.replace({'I полугодие': 1, 'II полугодие': 2})

#### Удаление данных

Удаление учебных годов (практически) без оценок

In [8]:
years_to_exclude = ['2023 - 2024', '2024 - 2025', '2025 - 2026', '2026 - 2027', '2027 - 2028']

df_new = df_new[~df_new['Учебный год'].isin(years_to_exclude)].copy()

In [9]:
df_new = df_new[~df_new['Учебная группа'].str.contains('22')].copy()

Удаление предметов без оценки

In [10]:
df_new = df_new.dropna(subset=['Оценка (без пересдач)', 'Оценка (успеваемость)'], how='all')

#### Заполнение пропусков в промежуточной аттестации

при условии, что есть итоговая оценка

In [11]:
df_new['Оценка (без пересдач)'] = df_new.apply(
    lambda row: 2 if pd.isna(row['Оценка (без пересдач)']) and not pd.isna(row['Оценка (успеваемость)']) else row['Оценка (без пересдач)'],
    axis=1
)

#### Добавление столбца с семестром

In [12]:
def calculate_semester(row):
    group_year = int(row['Учебная группа'].split('-')[1])
    start_year = int(row['Учебный год'].split(' - ')[0])
    course = (start_year % 100) - group_year + 1
    semester = course * 2 - 1 if row['Полугодие'] == 1 else course * 2
    return semester

df_new['Семестр'] = df_new.apply(calculate_semester, axis=1)

In [13]:
df_new['Программа'] = df_new['Учебная группа'].str.split('-').str[0]

In [14]:
df_new.head(2)

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость),Семестр,Программа
0,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,1,Инженерная компьютерная графика,Хорошо,Хорошо,1,БИВТ
1,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,1,Иностранный язык,Удовлетворительно,Удовлетворительно,1,БИВТ


In [15]:
disciplines = len(df_new['Дисциплина'].unique())
print(f'Количество уникальных дисциплин: {disciplines}')

Количество уникальных дисциплин: 658


In [16]:
programs = len(df_new['Программа'].unique())
print(f'Количество уникальных программ: {programs}')

Количество уникальных программ: 19


In [17]:
df = df_new
df = df.drop(columns=['Номер ЛД', 'Учебная группа', 'Уровень подготовки', 'Учебный год', 'Полугодие', 'Специальность/направление'])

In [18]:
df.head(1)

,hash,Дисциплина,Оценка (без пересдач),Оценка (успеваемость),Семестр,Программа
0,67a80fffd8d0294a596eda117d7e393c,Инженерная компьютерная графика,Хорошо,Хорошо,1,БИВТ


#### Кодирование оценок

In [19]:
df.replace({'зачтено': 5, 'Отлично': 5,
            'Хорошо': 4,
            'Удовлетворительно': 3,
            'Неудовлетворительно': 2, 'Неявка': 2, 'не зачтено': 2, 'Не допущен': 2,
            'Неявка по ув.причине': 0
            },
           inplace=True)

Так как по ТЗ все оставшиеся null у студента это 2, то заменим их:

In [20]:
df.fillna(2, inplace=True)

In [21]:
df.head(2)

,hash,Дисциплина,Оценка (без пересдач),Оценка (успеваемость),Семестр,Программа
0,67a80fffd8d0294a596eda117d7e393c,Инженерная компьютерная графика,4,4.0,1,БИВТ
1,67a80fffd8d0294a596eda117d7e393c,Иностранный язык,3,3.0,1,БИВТ


#### Создание столбцов для каждого направления

In [22]:
df_programs = pd.get_dummies(df, columns=['Программа'], prefix='', prefix_sep='')
df_programs = df_programs.drop(columns=['Дисциплина', 'Оценка (без пересдач)', 'Оценка (успеваемость)'])
df_programs = df_programs.astype({col: int for col in df_programs.columns[1:]})
df_programs = df_programs.drop_duplicates()

In [23]:
df_programs.head()

,hash,Семестр,ББИ,БИВТ,БИСТ,БЛГ,БМН,БМТ,БМТМ,БНМ,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,67a80fffd8d0294a596eda117d7e393c,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,67a80fffd8d0294a596eda117d7e393c,2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,67a80fffd8d0294a596eda117d7e393c,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,67a80fffd8d0294a596eda117d7e393c,4,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33,67a80fffd8d0294a596eda117d7e393c,5,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Создание столбцов для каждой дисциплины

Сопоставление итоговых оценок дисциплинам

In [24]:
pivot_df = df.pivot_table(index=['hash', 'Семестр'],
                          columns='Дисциплина',
                          values='Оценка (успеваемость)',
                          aggfunc='first'
                          ).reset_index()

pivot_df.columns.name = None
pivot_df.columns = [str(col) for col in pivot_df.columns]
pivot_df.fillna(0, inplace=True)
pivot_df = pivot_df.astype({col: int for col in pivot_df.columns[1:]})
pivot_df = pivot_df[pivot_df['Семестр'].isin([1, 2])].copy()
pivot_df = pivot_df.groupby('hash', as_index=False).sum()
pivot_df = pivot_df.drop(columns=['Семестр'])

In [25]:
pivot_df.head(2)

,hash,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,Автоматизация горных машин и установок,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Сопоставление промежуточных оценок дисциплинам

In [26]:
pivot_df_debts = df.pivot_table(index=['hash', 'Семестр'],
                          columns='Дисциплина',
                          values='Оценка (без пересдач)',
                          aggfunc='first'
                          ).reset_index()

pivot_df_debts.columns.name = None
pivot_df_debts.columns = [str(col) for col in pivot_df_debts.columns]
pivot_df_debts.fillna(0, inplace=True)
pivot_df_debts = pivot_df_debts.astype({col: int for col in pivot_df_debts.columns[1:]})
pivot_df_debts = pivot_df_debts[pivot_df_debts['Семестр'] == 3].copy()

In [27]:
pivot_df_debts.head()

,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
2,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,00083ac3c8aecc3a1cf66029173e56fa,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,002ce0ab2ee8101b0c19995bdf2b945d,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,002ed2297ad196e3b8a7e668f32d125b,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,0030a813467ddd7c36e9cd12d2b37943,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Подсчёт количества двоек в промежуточной аттестации в третьем семестре

In [28]:
df = pivot_df_debts

In [29]:
grades_columns = df.drop(['hash'], axis=1)

In [30]:
df['debts'] = (df == 2).sum(axis=1)
df

,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика,debts
2,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
8,00083ac3c8aecc3a1cf66029173e56fa,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,002ce0ab2ee8101b0c19995bdf2b945d,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18,002ed2297ad196e3b8a7e668f32d125b,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
24,0030a813467ddd7c36e9cd12d2b37943,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18075,ffbcf289933b630e44cc089ff4a873c8,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18081,ffc4f5381f1d58a1f6d4c541c71842bc,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18085,ffd3d5961c91b7211451654f570cd7ac,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18089,ffe63697b90c27f866b8424c43bc2fdf,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
df = pd.merge(pivot_df, df[['hash', 'debts']], on='hash', how='left')

In [32]:
df['Семестр'] = 3 # необходимый столбец для использования двух моделей в одном приложении

In [33]:
df

,hash,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,Автоматизация горных машин и установок,...,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика,debts,Семестр
0,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.0,3
1,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,3
2,002ce0ab2ee8101b0c19995bdf2b945d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,3
3,002ed2297ad196e3b8a7e668f32d125b,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.0,3
4,0030a813467ddd7c36e9cd12d2b37943,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3399,ffbcf289933b630e44cc089ff4a873c8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,3
3400,ffc4f5381f1d58a1f6d4c541c71842bc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,3
3401,ffd3d5961c91b7211451654f570cd7ac,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,3
3402,ffe63697b90c27f866b8424c43bc2fdf,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,3


In [34]:
df = df.dropna().copy()
df['debts'] = df['debts'].astype(int).copy()

In [35]:
df

,hash,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,Автоматизация горных машин и установок,...,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика,debts,Семестр
0,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,3
1,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,002ce0ab2ee8101b0c19995bdf2b945d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
3,002ed2297ad196e3b8a7e668f32d125b,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,3
4,0030a813467ddd7c36e9cd12d2b37943,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3399,ffbcf289933b630e44cc089ff4a873c8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3400,ffc4f5381f1d58a1f6d4c541c71842bc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3401,ffd3d5961c91b7211451654f570cd7ac,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
3402,ffe63697b90c27f866b8424c43bc2fdf,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


Смена порядка столбцов для удобства

In [36]:
cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]
print(cols)

['debts', 'Семестр', 'hash', 'BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений', 'BIM-технологии при проектировании горнодобывающих комплексов', 'CAD системы в горном производстве', 'CAD системы в подземном строительстве', 'Cертификация в горном деле', 'HR-системы организационного управления предприятием', 'Linux', 'Python для извлечения и обработки данных', 'Автоматизация горных машин и установок', 'Автоматизация письменного перевода и управление терминологией', 'Автоматизация технологических процессов', 'Автоматизированное проектирование машин', 'Автоматизированный электропривод машин и установок', 'Автомобили и тракторы', 'Академический английский, часть 1', 'Академический английский, часть 2', 'Академический английский. Часть 1', 'Алгоритмы анализа данных', 'Алгоритмы дискретной математики', 'Алгоритмы и структуры данных', 'Анализ данных и аналитика в принятии решений', 'Анализ данных на практике', 'Анализ и моделирование бизнес-процессов', 'Анализ

In [37]:
df = df[cols]

Сохранение всех дисциплин

In [38]:
lessons = ['Семестр'] + df.columns.to_list()[3:]
print(lessons)

['Семестр', 'BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений', 'BIM-технологии при проектировании горнодобывающих комплексов', 'CAD системы в горном производстве', 'CAD системы в подземном строительстве', 'Cертификация в горном деле', 'HR-системы организационного управления предприятием', 'Linux', 'Python для извлечения и обработки данных', 'Автоматизация горных машин и установок', 'Автоматизация письменного перевода и управление терминологией', 'Автоматизация технологических процессов', 'Автоматизированное проектирование машин', 'Автоматизированный электропривод машин и установок', 'Автомобили и тракторы', 'Академический английский, часть 1', 'Академический английский, часть 2', 'Академический английский. Часть 1', 'Алгоритмы анализа данных', 'Алгоритмы дискретной математики', 'Алгоритмы и структуры данных', 'Анализ данных и аналитика в принятии решений', 'Анализ данных на практике', 'Анализ и моделирование бизнес-процессов', 'Анализ точности маркшей

In [39]:
df.head(3)

,debts,Семестр,hash,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,4,3,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,3,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,3,002ce0ab2ee8101b0c19995bdf2b945d,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Подсчёт количества каждого вида оценок

In [40]:
subjects = df.apply(pd.Series.value_counts, axis=1)[[2, 3, 4, 5]].fillna(0)
subjects = subjects.astype(int)

#subjects['total'] = subjects[[2, 3, 4, 5]].sum(axis=1)
#subjects['total'] = pd.to_numeric(subjects['total'], errors='coerce').fillna(0)
subjects.head()

,2,3,4,5
0,0,4,2,6
1,0,1,2,4
2,0,2,2,3
3,0,3,3,2
4,1,3,4,4


In [41]:
df.head()

,debts,Семестр,hash,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,4,3,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,3,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,3,002ce0ab2ee8101b0c19995bdf2b945d,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,3,002ed2297ad196e3b8a7e668f32d125b,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,3,0030a813467ddd7c36e9cd12d2b37943,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
lessons = df.columns.to_list()[2:]
print(lessons)

['hash', 'BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений', 'BIM-технологии при проектировании горнодобывающих комплексов', 'CAD системы в горном производстве', 'CAD системы в подземном строительстве', 'Cертификация в горном деле', 'HR-системы организационного управления предприятием', 'Linux', 'Python для извлечения и обработки данных', 'Автоматизация горных машин и установок', 'Автоматизация письменного перевода и управление терминологией', 'Автоматизация технологических процессов', 'Автоматизированное проектирование машин', 'Автоматизированный электропривод машин и установок', 'Автомобили и тракторы', 'Академический английский, часть 1', 'Академический английский, часть 2', 'Академический английский. Часть 1', 'Алгоритмы анализа данных', 'Алгоритмы дискретной математики', 'Алгоритмы и структуры данных', 'Анализ данных и аналитика в принятии решений', 'Анализ данных на практике', 'Анализ и моделирование бизнес-процессов', 'Анализ точности маркшейдер

In [43]:
df.head(3)

,debts,Семестр,hash,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,4,3,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,3,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,3,002ce0ab2ee8101b0c19995bdf2b945d,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
df = df.astype({col: int for col in df.columns[3:]})

In [45]:
df.head(3)

,debts,Семестр,hash,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,4,3,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,3,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,3,002ce0ab2ee8101b0c19995bdf2b945d,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Объединение таблицы оценок с таблицей направлений

In [46]:
df_programs.head(1)

,hash,Семестр,ББИ,БИВТ,БИСТ,БЛГ,БМН,БМТ,БМТМ,БНМ,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,67a80fffd8d0294a596eda117d7e393c,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
df_merged = pd.merge(df, df_programs, on=['hash', 'Семестр'], how='inner')

In [48]:
df_merged.head()

,debts,Семестр,hash,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,4,3,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,3,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,3,002ce0ab2ee8101b0c19995bdf2b945d,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,3,002ed2297ad196e3b8a7e668f32d125b,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,3,0030a813467ddd7c36e9cd12d2b37943,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Удаление всех дисциплин

In [49]:
column_list = df_merged.columns.to_list()
column_list = column_list[:3] + column_list[-19:]
print(column_list)

['debts', 'Семестр', 'hash', 'ББИ', 'БИВТ', 'БИСТ', 'БЛГ', 'БМН', 'БМТ', 'БМТМ', 'БНМ', 'БНМТ', 'БПИ', 'БПМ', 'БТМО', 'БФЗ', 'БЭК', 'БЭН', 'БЭЭ', 'СГД', 'СНТС', 'СФП']


In [50]:
df_merged_cnt = df_merged[column_list]
df_merged_cnt = subjects.join(df_merged_cnt, how='inner')
df_merged_cnt.head(2)

,2,3,4,5,debts,Семестр,hash,ББИ,БИВТ,БИСТ,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,0,4,2,6,4,3,000006af6e40c8234a5af27896b7bba5,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,2,4,0,3,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
df_merged_cnt.to_csv('num_of_debts.csv', index=False)

In [52]:
df = pd.read_csv('num_of_debts.csv')

In [53]:
df # debts - количество долгов в следующем семестре для данного студента

,2,3,4,5,debts,Семестр,hash,ББИ,БИВТ,БИСТ,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,0,4,2,6,4,3,000006af6e40c8234a5af27896b7bba5,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,2,4,0,3,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,2,3,1,3,002ce0ab2ee8101b0c19995bdf2b945d,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,3,3,2,4,3,002ed2297ad196e3b8a7e668f32d125b,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,3,4,4,2,3,0030a813467ddd7c36e9cd12d2b37943,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3251,0,4,3,3,0,3,ffbcf289933b630e44cc089ff4a873c8,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3252,0,1,2,8,0,3,ffc4f5381f1d58a1f6d4c541c71842bc,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3253,0,1,1,6,1,3,ffd3d5961c91b7211451654f570cd7ac,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3254,0,1,1,5,0,3,ffe63697b90c27f866b8424c43bc2fdf,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [54]:
df = df.drop('hash', axis=1)

In [55]:
cols = df.columns.to_list()
cols = cols[:6]
print(cols)

['2', '3', '4', '5', 'debts', 'Семестр']


### Создание 2 датафреймов: с учётом направлений и без них

In [56]:
df_programs = df
df_no_programs = df[cols]

In [57]:
df_no_programs.head(2)

,2,3,4,5,debts,Семестр
0,0,4,2,6,4,3
1,0,1,2,4,0,3


## Обучение модели

In [58]:
results = df['debts'].to_numpy()
print(f'Минимальное количество двоек: {results.min()}')
print(f'Максимальное количество двоек: {results.max()}')
print(f'Среднее количество двоек: {results.mean():.3f}')

Минимальное количество двоек: 0
Максимальное количество двоек: 11
Среднее количество двоек: 1.738


### Обучение без учёта направления

#### Разделение на трейн-тест

In [59]:
X = df_no_programs.drop('debts', axis=1)
X = X.round().astype(int)
y = df_no_programs['debts']
y = y.astype(int)

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [61]:
X_train

,2,3,4,5,Семестр
1618,1,3,3,5,3
2786,0,2,3,2,3
1653,0,3,0,6,3
940,0,2,3,2,3
2093,0,4,1,6,3
...,...,...,...,...,...
1095,1,3,1,2,3
1130,1,6,1,4,3
1294,0,1,3,3,3
860,1,3,2,5,3


In [62]:
train_pool = Pool(X_train,
                  label=y_train)

test_pool = Pool(X_test,
                 label=y_test)

In [64]:
model = CatBoostRegressor(iterations=400, custom_metric='R2', learning_rate=0.1)
model.fit(train_pool, eval_set=[test_pool], verbose=200)

0:	learn: 2.1296408	test: 2.1206912	best: 2.1206912 (0)	total: 555us	remaining: 222ms
200:	learn: 2.0268828	test: 2.1846269	best: 2.1206912 (0)	total: 105ms	remaining: 104ms
399:	learn: 2.0032049	test: 2.2117579	best: 2.1206912 (0)	total: 214ms	remaining: 0us

bestTest = 2.120691188
bestIteration = 0

Shrink model to first 1 iterations.


In [65]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MAE: {mae:.4f}')
print(f'R2: {r2:.4f}')

MAE: 1.7372
R2: -0.0001


Видимо, без учёта направления всё совсем плохо

### Обучение с учётом направления

#### Разделение на трейн-тест

In [66]:
X = df_programs.drop('debts', axis=1)
X = X.round().astype(int)
y = df_programs['debts']
y = y.astype(int)

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [68]:
X_train

,2,3,4,5,Семестр,ББИ,БИВТ,БИСТ,БЛГ,БМН,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
1618,1,3,3,5,3,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2786,0,2,3,2,3,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1653,0,3,0,6,3,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
940,0,2,3,2,3,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2093,0,4,1,6,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,1,3,1,2,3,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1130,1,6,1,4,3,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1294,0,1,3,3,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
860,1,3,2,5,3,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
y_train

1618    0
2786    1
1653    0
940     3
2093    0
       ..
1095    2
1130    2
1294    4
860     0
3174    4
Name: debts, Length: 2442, dtype: int64

In [70]:
train_pool = Pool(X_train,
                  label=y_train)

test_pool = Pool(X_test,
                 label=y_test)

In [77]:
model = CatBoostRegressor(iterations=400, custom_metric='R2', learning_rate=0.1)
model.fit(train_pool, eval_set=[test_pool], verbose=200)

0:	learn: 2.1212661	test: 2.1127506	best: 2.1127506 (0)	total: 1.62ms	remaining: 646ms
200:	learn: 1.8693008	test: 2.0961044	best: 2.0762738 (51)	total: 349ms	remaining: 346ms
399:	learn: 1.7753160	test: 2.1262057	best: 2.0762738 (51)	total: 582ms	remaining: 0us

bestTest = 2.076273773
bestIteration = 51

Shrink model to first 52 iterations.


In [78]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MAE: {mae:.4f}')
print(f'R2: {r2:.4f}')

MAE: 1.6751
R2: 0.0414


С учётом направления не сильно лучше